In [9]:
import numpy as np
import time

def asian_call_mc(S0: float, K:float, r:float, sigma:float, T:float, num_steps:float, num_paths:float):
    """
    Prices an Asian call option using a simple Monte Carlo simulation

    Parameters:
    :param S0: Initial stock price.
    :param K: Strike price.
    :param r: Risk-free interest rate.
    :param sigma: Volatility of the underlying asset.
    :param T: Time to maturity in years.
    :param num_steps: Number of time steps to discretize the path.
    :param num_paths: Number of Monte Carlo simulation paths.

    Returns:
    tuple: (final_price, standard_error)
    """
    dt = T / num_steps

    # Generate random paths for the underlying Brownian motion
    Z = np.random.normal(0.0, 1.0, size=(num_paths // 2, num_steps))

    # Create Brownian motion increments
    dW = np.sqrt(dt) * Z

    # Simulate the full stock price paths for both sets of increments
    S = np.zeros((num_paths // 2, num_steps + 1))
    S[:, 0] = S0

    # Loop through time steps to build the paths
    for i in range(num_steps):
        S[:, i+1] = S[:, i] * np.exp((r - 0.5 * sigma**2) * dt + sigma * dW[:, i])

    # Calculate the arithmetic average for each path, use the trapezoidal rule for a more accurate integral approximation
    integral = dt * (0.5 * S[:, 0] + np.sum(S[:, 1:-1], axis=1) + 0.5 * S[:, -1])
    average_price = integral / T

    # Calculate the payoff for each path
    payoff = np.maximum(average_price - K, 0)

    # Discount the average payoff and calculate statistics
    discounted_payoff = np.exp(-r * T) * payoff
    final_price = np.mean(discounted_payoff)
    standard_error = np.std(discounted_payoff, ddof=1) / np.sqrt(num_paths // 2)

    return final_price, standard_error

if __name__ == "__main__":
    S0 = 1
    K = 1
    r = 0.05
    sigma = 0.20
    T = 1.0

    # Simulation parameters
    num_steps = 100
    num_paths = 100000

    print("--- Standard Monte Carlo for Asian Option ---")
    print(f"Parameters: S0={S0}, K={K}, r={r}, sigma={sigma}, T={T}")
    print(f"Simulation: {num_paths} paths, {num_steps} steps per path")

    # Calculate the price
    price, std_error = asian_call_mc(S0, K, r, sigma, T, num_steps, num_paths)

    print(f"\nEstimated Price: {price:.6f}")
    print(f"Standard Error:  {std_error:.6f}")

--- Standard Monte Carlo for Asian Option ---
Parameters: S0=1, K=1, r=0.05, sigma=0.2, T=1.0
Simulation: 100000 paths, 100 steps per path

Estimated Price: 0.057705
Standard Error:  0.000357
